In [1]:
import os

list_file = sorted(list(os.walk('./file/1.item_number'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

list_file

['crawler_1-item_error-item_error.xlsx']

In [2]:
file = list_file[0]

file

'crawler_1-item_error-item_error.xlsx'

In [3]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

In [4]:
input_ = pd.read_excel(f'./file/1.item_number/{file}',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
input_['Item Number'].tolist()

总数量：1


['204975916441']

In [5]:
a = 0

a

0

In [6]:
crawler_status = 'error'

crawler_status

'error'

In [7]:
import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

b = 0
while True:
    b += 1
    print(b)
    try:
        resp = requests.get(input_.loc[a, 'Url'],
                            headers=get_header(ua=False),
                            proxies=get_proxy(),
                            timeout=(10, 10))

        if resp.status_code == 200 or resp.status_code == 404:
            break
    except KeyboardInterrupt:
        break
    except:
        continue

resp

1


<Response [200]>

In [8]:
resp.status_code == 404

False

In [9]:
import json

json_ = json.loads(resp.text.split('$MC=(window.$MC||[]).concat(')[1].split(']}]})</script>')[0] + ']}]}')

json_

{'o': {'l': 1,
  'g': {'enableRumWeb': False,
   'disableUnsafeInline': False,
   'enableATCRedesign': 'false',
   'lazyLoaderType': 'NATIVE',
   'largerImageSize': 'FIXED_15',
   'signedIn': False,
   'epInfo': {'featureFlagDetails': {'enableRumWeb': {'value': False},
     'dwebDeliSSE': {'value': {'NORI_ENABLED': 'true'}, 'dropTag': 'server'},
     'enableSMEMerchBTF': {'value': False},
     'dwebAtfStreamingFeature': {'value': {'NORI_ENABLED': 'true',
       'DELI_ENABLED': 'false'}},
     'prefetchResourceFlag': {'value': {'VINODE_ENABLE_PREFETCH_RESOURCES': 'false'}},
     'dwebAtfSplitFeature': {'value': None},
     'enableOTPFeature': {'value': {'enableOTPFlow': 'false',
       'enableGuestOTPAuthFlow': False},
      'dropTag': 'browser'},
     'enableATCRedesign': {'value': {'enableATCLayerV3': 'false'}},
     'disableUnsafeInline': {'value': False},
     'enableAIImageDisclosure': {'value': False},
     'BXR_BUNDLED_TEST': {'value': {'ADS_SME_BTF_ENABLED': 'true',
       'NORI

In [10]:
if input_.loc[a, 'Url'].startswith('https://www.ebay.com/'):
    market, country, language, locale = 'EBAY-US', 'US', 'en', 'en-US'
elif input_.loc[a, 'Url'].startswith('https://www.ebay.de/'):
    market, country, language, locale = 'EBAY-DE', 'DE', 'de', 'de-DE'
elif input_.loc[a, 'Url'].startswith('https://www.ebay.co.uk/'):
    market, country, language, locale = 'EBAY-GB', 'GB', 'en', 'en-GB'
elif input_.loc[a, 'Url'].startswith('https://www.ebay.com.au/'):
    market, country, language, locale = 'EBAY-AU', 'AU', 'en', 'en-AU'
elif input_.loc[a, 'Url'].startswith('https://www.ebay.ca/'):
    market, country, language, locale = 'EBAY-ENCA', 'CA', 'en', 'en-CA'

print(market, country, language, locale)

json_['o']['g']['marketplaceId'].strip() != market or json_['o']['g']['country'].strip() != country or json_['o']['g']['language'].strip() != language or json_['o']['g']['locale'].strip() != locale or json_['o']['g']['itemId'].strip() != input_.loc[a, 'Item Number']

EBAY-US US en en-US


False

In [11]:
index_model = -1
for i in range(len(json_['o']['w'])):
    if 'model' in json_['o']['w'][i][2]:
        index_model = i
        break

index_model

0

In [12]:
index_model == -1

False

In [13]:
compatibility = json_['o']['w'][index_model][2]['model']['modules']['COMPATIBILITY_TABLE']['paginatedTable']['pagination']['pages'][-1] if 'COMPATIBILITY_TABLE' in json_['o']['w'][index_model][2]['model']['modules'] else ''
dict_compatibility = {} if compatibility == '' else {'api': compatibility['action']['URL'],
                                                     'page': int(compatibility['text'].strip()),
                                                     'data': {'scopedContext': {'catalogDetails': {'categoryId': compatibility['action']['params']['categoryId'],
                                                                                                   'itemId': compatibility['action']['params']['itemId'],
                                                                                                   'marketplaceId': compatibility['action']['params']['marketplaceId'],
                                                                                                   'sellerName': compatibility['action']['params']['sellerName']}}}}

dict_compatibility

{}

In [14]:
dict_specific = {}
dict_item = json_['o']['w'][index_model][2]['model']['modules']['ABOUT_THIS_ITEM']['sections']['features']['dataItems'] if 'ABOUT_THIS_ITEM' in json_['o']['w'][index_model][2]['model']['modules'] else {}
for i, (key, value) in zip(range(len(dict_item.items())), dict_item.items()):
    dict_specific[str(i)] = {value['labels'][0]['textSpans'][0]['text'].strip(): value['values'][0]['textualDisplays'][0]['textSpans'][0]['text'].strip() if 'textualDisplays' in value['values'][0] else value['values'][0]['textSpans'][0]['text'].strip()}

dict_specific

{}

In [15]:
description_url = json_['o']['w'][index_model][2]['model']['modules']['ITEM_DESCRIPTION_MIN_VIEW_MODULE']['sections'][0]['dataItems'][0]['action']['URL'].strip() if 'dataItems' in json_['o']['w'][index_model][2]['model']['modules']['ITEM_DESCRIPTION_MIN_VIEW_MODULE']['sections'][0] else json_['o']['w'][index_model][2]['model']['modules']['ITEM_DESCRIPTION_MIN_VIEW_MODULE']['sections'][0]['action']['URL'].strip()

description_url

'https://vi.vipr.ebaydesc.com/itmdesc/204975916441?t=0&category=47121&seller=oghautoparts&excSoj=1&ver=0&excTrk=1&lsite=0&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1'

In [16]:
title = json_['o']['w'][index_model][2]['model']['modules']['TITLE']['mainTitle']['textSpans'][0]['text'].strip()

title

'07 Acura TL  Type S Air intake resonator chamber oem'

In [17]:
subtitle = json_['o']['w'][index_model][2]['model']['modules']['TITLE']['subTitle']['textSpans'][0]['text'].strip() if 'subTitle' in json_['o']['w'][index_model][2]['model']['modules']['TITLE'] else ''

subtitle

''

In [18]:
price = json_['o']['w'][index_model][2]['model']['modules']['BUY_BOX']['binModel']['price']['textSpans'][0]['text'].strip() if 'binModel' in json_['o']['w'][index_model][2]['model']['modules']['BUY_BOX'] else json_['o']['w'][0][2]['model']['modules']['BUY_BOX']['bidPrice']['textSpans'][0]['text'].strip()

price

'US $80.00'

In [19]:
available, sold = '', ''
list_quantity = json_['o']['w'][index_model][2]['model']['modules']['QUANTITY']['availabilitySignal']['textSpans'] if 'QUANTITY' in json_['o']['w'][index_model][2]['model']['modules'] else []
for quantity in list_quantity:
    if input_.loc[a, 'Url'].startswith('https://www.ebay.de/'):
        if 'verfügbar' in quantity['text']:
            available = quantity['text'].strip()
        elif 'verkauft' in quantity['text']:
            sold = quantity['text'].strip()
    else:
        if 'available' in quantity['text']:
            available = quantity['text'].strip()
        elif 'sold' in quantity['text']:
            sold = quantity['text'].strip()

available, sold

('', '')

In [20]:
dict_src = {}
list_media = json_['o']['w'][index_model][2]['model']['modules']['PICTURE']['mediaList'] if 'mediaList' in json_['o']['w'][index_model][2]['model']['modules']['PICTURE'] else []
for i, media in zip(range(len(list_media)), list_media):
    dict_src[str(i)] = media['image']['originalImg']['URL'].strip()

dict_src

{'0': 'https://i.ebayimg.com/images/g/K-gAAOSwQ-xm2LI5/s-l500.jpg',
 '1': 'https://i.ebayimg.com/images/g/uaIAAOSwvIZm2LI5/s-l500.jpg',
 '2': 'https://i.ebayimg.com/images/g/sIIAAOSw9KFm2LI5/s-l500.jpg'}

In [21]:
df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                         'Item Number': input_.loc[a, 'Item Number'],
                         'Title': title,
                         'Subtitle': subtitle,
                         'Price': price,
                         'Sold': sold,
                         'Available': available,
                         'Picture': '',
                         'Url': input_.loc[a, 'Url'],
                         'Description Url': description_url,
                         'Json_Src': json.dumps(dict_src),
                         'Json_Specific': json.dumps(dict_specific),
                         'Json_Compatibility': json.dumps(dict_compatibility)}])

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

output_correct

,No,Item Number,Title,Subtitle,Price,Sold,Available,Picture,Url,Description Url,Json_Src,Json_Specific,Json_Compatibility
0,970,204975916441,07 Acura TL Type S Air intake resonator chamb...,,US $80.00,,,,https://www.ebay.com/itm/204975916441?_ul=US&_...,https://vi.vipr.ebaydesc.com/itmdesc/204975916...,"{""0"": ""https://i.ebayimg.com/images/g/K-gAAOSw...",{},{}


In [22]:
crawler_status = 'ok'

crawler_status

'ok'

In [23]:
if crawler_status == 'error':
    output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

output_error

""


In [24]:
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
    output_correct.to_excel('./test_item.xlsx', index=False)

output_correct

,No,Item Number,Title,Subtitle,Price,Sold,Available,Picture,Url,Description Url,Json_Src,Json_Specific,Json_Compatibility
0,970,204975916441,07 Acura TL Type S Air intake resonator chamb...,,US $80.00,,,,https://www.ebay.com/itm/204975916441?_ul=US&_...,https://vi.vipr.ebaydesc.com/itmdesc/204975916...,"{""0"": ""https://i.ebayimg.com/images/g/K-gAAOSw...",{},{}


In [25]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel('./test_item_error.xlsx', index=False)

output_error

""
